In [5]:
import numpy as np
array = [[1,2,3],[4,5,6]]
np.shape(np.array(array))

(2, 3)

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### Preparing data

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data", # The location of the downlode file
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [12]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [16]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [Batch size, Channel, Height, Width]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [Batch size, Channel, Height, Width]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### Creating Models

In [17]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [18]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # input is 28 * 28 picture, output is 512 
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x) # flatten 28 * 28 picture to a feature vector
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizing the Model Parameters

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.317862  [    0/60000]
loss: 2.296541  [ 6400/60000]
loss: 2.284981  [12800/60000]
loss: 2.267387  [19200/60000]
loss: 2.242686  [25600/60000]
loss: 2.236547  [32000/60000]
loss: 2.231990  [38400/60000]
loss: 2.203932  [44800/60000]
loss: 2.206469  [51200/60000]
loss: 2.169724  [57600/60000]
Test Error: 
 Accuracy: 45.2%, Avg loss: 2.162004 

Epoch 2
-------------------------------
loss: 2.178958  [    0/60000]
loss: 2.162686  [ 6400/60000]
loss: 2.112405  [12800/60000]
loss: 2.122524  [19200/60000]
loss: 2.065174  [25600/60000]
loss: 2.025076  [32000/60000]
loss: 2.042681  [38400/60000]
loss: 1.965554  [44800/60000]
loss: 1.974235  [51200/60000]
loss: 1.900991  [57600/60000]
Test Error: 
 Accuracy: 54.3%, Avg loss: 1.894250 

Epoch 3
-------------------------------
loss: 1.932208  [    0/60000]
loss: 1.893456  [ 6400/60000]
loss: 1.782105  [12800/60000]
loss: 1.821259  [19200/60000]
loss: 1.709157  [25600/60000]
loss: 1.669086  [32000/600

### Saving Models

In [26]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### Loading Models

In [28]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

### Make predictions

In [77]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    print(pred[0])
    print(pred[0].argmax(0)) # argmax(0) 0 means the dimension, which is the row
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor([-2.0346, -2.6571, -0.9089, -1.9110, -0.8472,  2.5418, -1.0854,  2.7738,
         1.9279,  3.2508])
tensor(9)
Predicted: "Ankle boot", Actual: "Ankle boot"


torch.Size([1, 28, 28])